Данный код, можно было написать в обычный файл main.py, но был выбран nbg файла ipynb для удобства написании кода и отлеживание процессов тому кто пишет код итому кто читает. отдельные ячейки с кодами, позволяют видет что делает конкретный блок кода и как влияет на данные. все ячейки кода запускаются комбиациями клавиш ctrl+f9

импортирование библиотек


In [276]:
import pandas as pd # For structured data manipulation and analysis
import re # For regular expression operations (text pattern matching)
from bs4 import BeautifulSoup # For parsing HTML and XML documents (web scraping)
import json # For working with JSON data
import csv # For working with CSV files
import requests
from loguru import logger
from time import sleep
print("Common data parsing libraries imported successfully.")

Common data parsing libraries imported successfully.


загрузка данных полученных из сайта https://www.list-org.com/
отсортированных по ОКВЭД IT кодам


In [ ]:
df = pd.read_excel("data/raw/list_page_1.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Ссылка на www.list-org.com  0 non-null      float64
 1   Наименование                50 non-null     object 
 2   Юридическое наименование    50 non-null     object 
 3   Статус                      50 non-null     object 
 4   Руководитель                50 non-null     object 
 5   ИНН                         50 non-null     int64  
 6   ОГРН                        50 non-null     int64  
 7   Телефон (один из)           50 non-null     object 
 8   E-mail                      50 non-null     object 
 9   Сайт                        50 non-null     object 
 10  Юридический адрес           50 non-null     object 
 11  Сотрудников                 50 non-null     int64  
 12  Выручка (Ф2.2110)           50 non-null     int64  
 13  Прибыль (Ф2.2400)           50 non-nu

добавление остальных данных

In [ ]:
for i in range(2,4):
  df = pd.concat([df, pd.read_excel(f'data/raw/list_page_{i}.xlsx')], ignore_index=True)

In [279]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Ссылка на www.list-org.com  0 non-null      float64
 1   Наименование                131 non-null    object 
 2   Юридическое наименование    131 non-null    object 
 3   Статус                      131 non-null    object 
 4   Руководитель                131 non-null    object 
 5   ИНН                         131 non-null    int64  
 6   ОГРН                        131 non-null    int64  
 7   Телефон (один из)           131 non-null    object 
 8   E-mail                      131 non-null    object 
 9   Сайт                        131 non-null    object 
 10  Юридический адрес           131 non-null    object 
 11  Сотрудников                 131 non-null    int64  
 12  Выручка (Ф2.2110)           129 non-null    float64
 13  Прибыль (Ф2.2400)           130 non

убираем  ненужыне столбцы (по требованию)

In [280]:
df = df.drop(columns=[
    'Ссылка на www.list-org.com',
    'Юридическое наименование',
    'Статус',
    'ОГРН',
    'Руководитель',
    'Прибыль (Ф2.2400)',
    'Стоимость (Ф1.1300)',
    'В избранном',
    'Описание',
    ])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Наименование       131 non-null    object 
 1   ИНН                131 non-null    int64  
 2   Телефон (один из)  131 non-null    object 
 3   E-mail             131 non-null    object 
 4   Сайт               131 non-null    object 
 5   Юридический адрес  131 non-null    object 
 6   Сотрудников        131 non-null    int64  
 7   Выручка (Ф2.2110)  129 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 8.3+ KB


переименовываем название столбцов (по требованию)

In [281]:
df = df.rename(columns={
  'Наименование': 'name',
  'ИНН': 'inn',
  'Телефон (один из)': 'phone',
  'E-mail': 'email',
  'Сайт': 'site',
  'Юридический адрес': 'address',
  'Сотрудников': 'employees',
  'Выручка (Ф2.2110)': 'revenue',
})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       131 non-null    object 
 1   inn        131 non-null    int64  
 2   phone      131 non-null    object 
 3   email      131 non-null    object 
 4   site       131 non-null    object 
 5   address    131 non-null    object 
 6   employees  131 non-null    int64  
 7   revenue    129 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 8.3+ KB


добавляем недостающие столбцы sourse и revenue_year

In [282]:
df['revenue_year'] = 2024
df['source'] = 'list-org.com'
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          131 non-null    object 
 1   inn           131 non-null    int64  
 2   phone         131 non-null    object 
 3   email         131 non-null    object 
 4   site          131 non-null    object 
 5   address       131 non-null    object 
 6   employees     131 non-null    int64  
 7   revenue       129 non-null    float64
 8   revenue_year  131 non-null    int64  
 9   source        131 non-null    object 
dtypes: float64(1), int64(3), object(6)
memory usage: 10.4+ KB


нужно переделать address в region, т.е. добаавить region и удалить address (по треббованию они не нужны)

In [283]:
df.address

,address
0,"117574, Г.МОСКВА, ВНУТРИГОРОДСКАЯ ТЕРРИТОРИЯ Г..."
1,"125284, Г.МОСКВА, ВНУТРИГОРОДСКАЯ ТЕРРИТОРИЯ Г..."
2,"344038, РОСТОВСКАЯ ОБЛАСТЬ, Г. РОСТОВ-НА-ДОНУ,..."
3,"394036, ВОРОНЕЖСКАЯ ОБЛАСТЬ, Г ВОРОНЕЖ, УЛ СТУ..."
4,"129226, Г.МОСКВА, УЛ. ДОКУКИНА, Д. 16, СТР. 1,..."
...,...
126,"299020, Г.СЕВАСТОПОЛЬ , УЛ ХРУСТАЛЕВА, ЗД. 74А"
127,"117342, Г.Москва, ВНУТРИГОРОДСКАЯ ТЕРРИТОРИЯ Г..."
128,"620026, СВЕРДЛОВСКАЯ ОБЛАСТЬ, ГОРОД ЕКАТЕРИНБУ..."
129,"107113, Г.МОСКВА, УЛ. 3-Я РЫБИНСКАЯ, Д. 17, КО..."


Из данных видно, что нужные города/области прописаны после шестизначных чисел

In [284]:
df['region'] = df.address.str.split(',').str[1].str.strip().str.title()
df = df.drop(columns=['address'])
df.region

,region
0,Г.Москва
1,Г.Москва
2,Ростовская Область
3,Воронежская Область
4,Г.Москва
...,...
126,Г.Севастополь
127,Г.Москва
128,Свердловская Область
129,Г.Москва


In [285]:
df.head()

,name,inn,phone,email,site,employees,revenue,revenue_year,source,region
0,"АО ""АТГС""",7723011060,+7 (495) 660-08-02,"atgs@atgs.ru, isheva@atgs.ru, lapegina@atgs.ru...",http://atgs.ru,136,1258580.0,2024,list-org.com,Г.Москва
1,"ООО ""МЕТТЭМ-М""",7713011008,480-92-92,4800561@mail.ru,http://mettem-m.ru,160,2102140.0,2024,list-org.com,Г.Москва
2,"ООО ""НПП ""ЮГПРОМАВТОМАТИЗАЦИЯ"" или ООО ""НПП ""Ю...",6165000652,+7 (863) 245-45-09,info@ugpa.ru,www.ugpa.ru,389,1338750.0,2024,list-org.com,Ростовская Область
3,"ФАУ ""ГНИИИ ПТЗИ ФСТЭК России""",3666088317,+7 (473) 253-99-85,"gniii@fstec.ru, ptzi2015@yandex.ru",www.rosdornii-vrn.ru,242,1368440.0,2024,list-org.com,Воронежская Область
4,"АО ""АЛАДДИН Р.Д.""",7719165935,+7 (495) 223-00-01,aladdin@aladdin-rd.ru,www.aladdin-rd.ru,185,1109860.0,2024,list-org.com,Г.Москва


Убираем номера, где пропущены "+7". вместо них ставвим Nan.

In [286]:
phone_mask = df.phone.str.startswith('+7')
df.loc[~phone_mask, "phone"] = pd.NA
df.phone

,phone
0,+7 (495) 660-08-02
1,<NA>
2,+7 (863) 245-45-09
3,+7 (473) 253-99-85
4,+7 (495) 223-00-01
...,...
126,+7 (0692) 93-30-00
127,+7 (495) 748-01-23
128,+7 (343) 355-81-00
129,+7 (800) 500-48-05


нужно удалить данные с пустыми данными выручки (revenue)

In [287]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          131 non-null    object 
 1   inn           131 non-null    int64  
 2   phone         128 non-null    object 
 3   email         131 non-null    object 
 4   site          131 non-null    object 
 5   employees     131 non-null    int64  
 6   revenue       129 non-null    float64
 7   revenue_year  131 non-null    int64  
 8   source        131 non-null    object 
 9   region        131 non-null    object 
dtypes: float64(1), int64(3), object(6)
memory usage: 10.4+ KB


In [288]:
df = df[df.revenue.notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129 entries, 0 to 130
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          129 non-null    object 
 1   inn           129 non-null    int64  
 2   phone         126 non-null    object 
 3   email         129 non-null    object 
 4   site          129 non-null    object 
 5   employees     129 non-null    int64  
 6   revenue       129 non-null    float64
 7   revenue_year  129 non-null    int64  
 8   source        129 non-null    object 
 9   region        129 non-null    object 
dtypes: float64(1), int64(3), object(6)
memory usage: 11.1+ KB


теперь на сайте https://www.rusprofile.ru ищем оставшие информации для каждой компании okved_main и description
с помощью парсинга


In [290]:
df.head()

,name,inn,phone,email,site,employees,revenue,revenue_year,source,region
0,"АО ""АТГС""",7723011060,+7 (495) 660-08-02,"atgs@atgs.ru, isheva@atgs.ru, lapegina@atgs.ru...",http://atgs.ru,136,1258580.0,2024,list-org.com,Г.Москва
1,"ООО ""МЕТТЭМ-М""",7713011008,<NA>,4800561@mail.ru,http://mettem-m.ru,160,2102140.0,2024,list-org.com,Г.Москва
2,"ООО ""НПП ""ЮГПРОМАВТОМАТИЗАЦИЯ"" или ООО ""НПП ""Ю...",6165000652,+7 (863) 245-45-09,info@ugpa.ru,www.ugpa.ru,389,1338750.0,2024,list-org.com,Ростовская Область
3,"ФАУ ""ГНИИИ ПТЗИ ФСТЭК России""",3666088317,+7 (473) 253-99-85,"gniii@fstec.ru, ptzi2015@yandex.ru",www.rosdornii-vrn.ru,242,1368440.0,2024,list-org.com,Воронежская Область
4,"АО ""АЛАДДИН Р.Д.""",7719165935,+7 (495) 223-00-01,aladdin@aladdin-rd.ru,www.aladdin-rd.ru,185,1109860.0,2024,list-org.com,Г.Москва


In [291]:
def fetch_html(url: str) -> str:
    """
    Забираем HTML страницы через requests.
    """
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding": "gzip, deflate, br",
    }

    try:
        response = requests.get(url, headers=headers)
        logger.info("GET {} status={}", url, response.status_code)
        response.raise_for_status()
        text = response.text
        logger.debug("Received {} chars of HTML for page {}", len(text), url)
        return text
    except Exception as e:
        logger.exception("Ошибка при запросе к сайту {}: {}", url, e)
        return ""

In [292]:
def get_okved_main(html):
  soup = BeautifulSoup(html, 'lxml')
  try:
    logger.debug("Поиск ОКВЭД")
    col = soup.find("div", class_="rightcol")
    row = col.find("div", class_="company-row")
    result = row.find("span", class_="company-info__text")
    logger.debug("Найдено!")
  except Exception as e:
    logger.exception("Ошибка при получении ОКВЭД: не найдено!")
    return pd.NA
  return result.get_text(strip=True)

In [293]:
get_okved_main(fetch_html(f'https://www.rusprofile.ru/search?query={inn}'))

2025-12-11 14:51:18.777 | INFO     | __main__:fetch_html:18 - GET https://www.rusprofile.ru/search?query=7723011060 status=200
2025-12-11 14:51:18.779 | DEBUG    | __main__:fetch_html:21 - Received 260282 chars of HTML for page https://www.rusprofile.ru/search?query=7723011060
2025-12-11 14:51:18.883 | DEBUG    | __main__:get_okved_main:4 - Поиск ОКВЭД
2025-12-11 14:51:18.885 | DEBUG    | __main__:get_okved_main:8 - Найдено!


'Деятельность по подготовке компьютерных систем к эксплуатации(62.02.4)'

In [294]:
df['okved_main'] = pd.NA # создаем столбец okved_main

In [295]:
df = df.reset_index(drop=True) # обновляем индексы

In [296]:
inns = df.inn.copy()
for i in range(df.shape[0]):
  okved_main = get_okved_main(fetch_html(f'https://www.rusprofile.ru/search?query={inns.loc[i]}'))
  df.at[i, 'okved_main'] = okved_main
  sleep(2)

2025-12-11 14:51:20.448 | INFO     | __main__:fetch_html:18 - GET https://www.rusprofile.ru/search?query=7723011060 status=200
2025-12-11 14:51:20.450 | DEBUG    | __main__:fetch_html:21 - Received 259572 chars of HTML for page https://www.rusprofile.ru/search?query=7723011060
2025-12-11 14:51:20.553 | DEBUG    | __main__:get_okved_main:4 - Поиск ОКВЭД
2025-12-11 14:51:20.556 | DEBUG    | __main__:get_okved_main:8 - Найдено!
2025-12-11 14:51:24.055 | INFO     | __main__:fetch_html:18 - GET https://www.rusprofile.ru/search?query=7713011008 status=200
2025-12-11 14:51:24.058 | DEBUG    | __main__:fetch_html:21 - Received 258713 chars of HTML for page https://www.rusprofile.ru/search?query=7713011008
2025-12-11 14:51:25.422 | DEBUG    | __main__:get_okved_main:4 - Поиск ОКВЭД
2025-12-11 14:51:25.425 | DEBUG    | __main__:get_okved_main:8 - Найдено!
2025-12-11 14:51:28.931 | INFO     | __main__:fetch_html:18 - GET https://www.rusprofile.ru/search?query=6165000652 status=200
2025-12-11 14:5

In [297]:
df.head()

,name,inn,phone,email,site,employees,revenue,revenue_year,source,region,okved_main
0,"АО ""АТГС""",7723011060,+7 (495) 660-08-02,"atgs@atgs.ru, isheva@atgs.ru, lapegina@atgs.ru...",http://atgs.ru,136,1258580.0,2024,list-org.com,Г.Москва,Деятельность по подготовке компьютерных систем...
1,"ООО ""МЕТТЭМ-М""",7713011008,<NA>,4800561@mail.ru,http://mettem-m.ru,160,2102140.0,2024,list-org.com,Г.Москва,"Деятельность по планированию, проектированию к..."
2,"ООО ""НПП ""ЮГПРОМАВТОМАТИЗАЦИЯ"" или ООО ""НПП ""Ю...",6165000652,+7 (863) 245-45-09,info@ugpa.ru,www.ugpa.ru,389,1338750.0,2024,list-org.com,Ростовская Область,Деятельность по подготовке компьютерных систем...
3,"ФАУ ""ГНИИИ ПТЗИ ФСТЭК России""",3666088317,+7 (473) 253-99-85,"gniii@fstec.ru, ptzi2015@yandex.ru",www.rosdornii-vrn.ru,242,1368440.0,2024,list-org.com,Воронежская Область,Деятельность в области защиты информации и обе...
4,"АО ""АЛАДДИН Р.Д.""",7719165935,+7 (495) 223-00-01,aladdin@aladdin-rd.ru,www.aladdin-rd.ru,185,1109860.0,2024,list-org.com,Г.Москва,"Производство средств защиты информации, а такж..."


In [298]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          129 non-null    object 
 1   inn           129 non-null    int64  
 2   phone         126 non-null    object 
 3   email         129 non-null    object 
 4   site          129 non-null    object 
 5   employees     129 non-null    int64  
 6   revenue       129 non-null    float64
 7   revenue_year  129 non-null    int64  
 8   source        129 non-null    object 
 9   region        129 non-null    object 
 10  okved_main    121 non-null    object 
dtypes: float64(1), int64(3), object(7)
memory usage: 11.2+ KB


8 из 129 ссылок при переходе на сайт отображала не карточку компании, а спиоск компаний или людей, что я считаю это логическая ошибка самого сайта, так как из списка ИНН все разные и разумеется нелогично.

оставляем только те компании, в которых есть okved_main

In [301]:
df = df[df.okved_main.notna()]
df = df.reset_index(drop=True) # обновляем индексы
df

,name,inn,phone,email,site,employees,revenue,revenue_year,source,region,okved_main
0,"АО ""АТГС""",7723011060,+7 (495) 660-08-02,"atgs@atgs.ru, isheva@atgs.ru, lapegina@atgs.ru...",http://atgs.ru,136,1258580.0,2024,list-org.com,Г.Москва,Деятельность по подготовке компьютерных систем...
1,"ООО ""МЕТТЭМ-М""",7713011008,<NA>,4800561@mail.ru,http://mettem-m.ru,160,2102140.0,2024,list-org.com,Г.Москва,"Деятельность по планированию, проектированию к..."
2,"ООО ""НПП ""ЮГПРОМАВТОМАТИЗАЦИЯ"" или ООО ""НПП ""Ю...",6165000652,+7 (863) 245-45-09,info@ugpa.ru,www.ugpa.ru,389,1338750.0,2024,list-org.com,Ростовская Область,Деятельность по подготовке компьютерных систем...
3,"ФАУ ""ГНИИИ ПТЗИ ФСТЭК России""",3666088317,+7 (473) 253-99-85,"gniii@fstec.ru, ptzi2015@yandex.ru",www.rosdornii-vrn.ru,242,1368440.0,2024,list-org.com,Воронежская Область,Деятельность в области защиты информации и обе...
4,"АО ""АЛАДДИН Р.Д.""",7719165935,+7 (495) 223-00-01,aladdin@aladdin-rd.ru,www.aladdin-rd.ru,185,1109860.0,2024,list-org.com,Г.Москва,"Производство средств защиты информации, а такж..."
...,...,...,...,...,...,...,...,...,...,...,...
116,"ООО ""СЕВСТАР ИСПС""",9201003689,+7 (0692) 93-30-00,"help@sevstar.net,job@sevstar.net",https://sevstar.net,352,612271.0,2024,list-org.com,Г.Севастополь,"Деятельность, связанная с использованием вычис..."
117,"ООО ""ЛАБОРАТОРИЯ ТЕХНОЛОГИЙ""",7706444191,+7 (495) 748-01-23,teh@1cbit.ru,https://www.1cbit.ru,139,1642290.0,2024,list-org.com,Г.Москва,"Деятельность, связанная с использованием вычис..."
118,"ООО ""СКБ ЛАБ""",6685124446,+7 (343) 355-81-00,hr@skblab.ru,https://skblab.ru,198,611637.0,2024,list-org.com,Свердловская Область,Разработка компьютерного программного обеспече...
119,"ООО ""КОГНИТИВ РОБОТИКС""",9718131366,+7 (800) 500-48-05,info@cognitive.ru,https://cognitive.ru,157,180583.0,2024,list-org.com,Г.Москва,Разработка компьютерного программного обеспече...


так же в требовании есть требование нормализовать данные. из всех данных, такое прожно сделать со столбцом employees. Но для нормализованных данных, создадим отдельный столбец


In [303]:
max_employees = df.employees.max()
min_employees = df.employees.min()
print(f'Максимальное число сотрудников: {max_employees}')
print(f'Минимальное число сотрудников: {min_employees}')
df['employees_norm'] = (df.employees - min_employees) / (max_employees - min_employees)
df['employees_norm']

Максимальное число сотрудников: 493
Минимальное число сотрудников: 100


,employees_norm
0,0.091603
1,0.152672
2,0.735369
3,0.361323
4,0.216285
...,...
116,0.641221
117,0.099237
118,0.249364
119,0.145038


меняем порядок столбцов как в ТЗ

In [306]:
cols = ['inn','name','employees','employees_norm','okved_main','source','revenue_year','revenue','site','region', 'phone','email']
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   inn             121 non-null    int64  
 1   name            121 non-null    object 
 2   employees       121 non-null    int64  
 3   employees_norm  121 non-null    float64
 4   okved_main      121 non-null    object 
 5   source          121 non-null    object 
 6   revenue_year    121 non-null    int64  
 7   revenue         121 non-null    float64
 8   site            121 non-null    object 
 9   region          121 non-null    object 
 10  phone           119 non-null    object 
 11  email           121 non-null    object 
dtypes: float64(2), int64(3), object(7)
memory usage: 11.5+ KB


In [307]:
df.columns

Index(['inn', 'name', 'employees', 'employees_norm', 'okved_main', 'source',
       'revenue_year', 'revenue', 'site', 'region', 'phone', 'email'],
      dtype='object')

source добавляю еще один сайт, откуда я брал данные

In [310]:
df.source = df.source + ', rusprofile.ru'
df.source

,source
0,"list-org.com, rusprofile.ru"
1,"list-org.com, rusprofile.ru"
2,"list-org.com, rusprofile.ru"
3,"list-org.com, rusprofile.ru"
4,"list-org.com, rusprofile.ru"
...,...
116,"list-org.com, rusprofile.ru"
117,"list-org.com, rusprofile.ru"
118,"list-org.com, rusprofile.ru"
119,"list-org.com, rusprofile.ru"


ФИНАЛ: выгружаем обработанные данные в итоговый файл data.csv

In [ ]:
df.to_csv('data/companies.csv', index=False)